# Singular Value Decomposition (SVD)


Singular Value Decomposition (SVD), a matrix A can be decomposed as:

A
=
U
Σ
V^T

If the dimensions of matrix A are n x m, then the matrix U has dimensions n x n, and the matrix V has dimensions m x m.

In [68]:
import numpy as np
from scipy.linalg import svd
from numpy import diag
from numpy import zeros

In [69]:
A = np.random.randn(100,49) # 100 , 7x7

In [70]:
U,S,VT = svd(A)

In [71]:
print(U.shape) # number of rows of matrix A
print(S.shape) # vector
print(VT.shape) # number of columns of matrix A

(100, 100)
(49,)
(49, 49)


In [72]:
print(S)

[16.66948374 16.26058772 15.91051372 15.49175028 15.01699582 14.288657
 14.03816858 13.56007781 13.20840704 12.93129246 12.5151939  12.38674661
 12.17040371 11.77362705 11.59138644 11.36231321 11.25954374 10.82437418
 10.52823214 10.17291352 10.09484535  9.88248307  9.31951207  9.24621311
  9.1184015   8.82252891  8.7491829   8.29230165  8.24463817  8.07710917
  7.88662826  7.50277807  7.33277529  7.17673146  6.92550049  6.90762055
  6.58002202  6.09856546  5.97099679  5.80532225  5.68705672  5.39088611
  5.21101729  5.07985222  4.6485425   4.28593923  4.15015616  3.70932091
  3.29744046]


In [73]:
# to make the size of S same to the size of A

sigma = zeros((A.shape[0], A.shape[1]))
sigma[:A.shape[1],:A.shape[1]] = diag(S)

In [74]:
print(sigma)
print(sigma.shape)

[[16.66948374  0.          0.         ...  0.          0.
   0.        ]
 [ 0.         16.26058772  0.         ...  0.          0.
   0.        ]
 [ 0.          0.         15.91051372 ...  0.          0.
   0.        ]
 ...
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]]
(100, 49)


In [75]:
A

array([[ 1.03764425,  1.27624974, -0.7175942 , ...,  0.7708957 ,
         0.2922959 , -0.30572119],
       [-1.35875075,  0.7537294 ,  1.56300806, ..., -1.66601751,
        -0.56285249,  0.43151994],
       [ 0.59750042,  1.8722858 ,  0.89831191, ...,  1.24310711,
        -0.31392677,  0.43437198],
       ...,
       [ 0.69978889, -0.40526957, -0.59600642, ...,  0.07875108,
         0.311613  , -1.02741657],
       [ 0.04085183, -1.04173635,  0.5613235 , ..., -1.02425913,
         0.32118132, -0.82544677],
       [ 0.04480918, -1.40731216,  0.58221369, ...,  1.12580578,
         0.23203362, -0.95208381]])

In [76]:
B = U.dot(sigma.dot(VT)) # Or B = U @ sigma @ VT
B #we can get A from U,S,VT (B==A)

array([[ 1.03764425,  1.27624974, -0.7175942 , ...,  0.7708957 ,
         0.2922959 , -0.30572119],
       [-1.35875075,  0.7537294 ,  1.56300806, ..., -1.66601751,
        -0.56285249,  0.43151994],
       [ 0.59750042,  1.8722858 ,  0.89831191, ...,  1.24310711,
        -0.31392677,  0.43437198],
       ...,
       [ 0.69978889, -0.40526957, -0.59600642, ...,  0.07875108,
         0.311613  , -1.02741657],
       [ 0.04085183, -1.04173635,  0.5613235 , ..., -1.02425913,
         0.32118132, -0.82544677],
       [ 0.04480918, -1.40731216,  0.58221369, ...,  1.12580578,
         0.23203362, -0.95208381]])

In [77]:
A_ = U[:,:16] @ sigma[:16] @ VT[:,:16]

In [78]:
A_.shape

(100, 16)

# Perform low-rank factorization on MNIST

In [79]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [80]:
train_dataset = MNIST(root="",download = True, train = True, transform= transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]))

In [81]:
train_data = train_dataset.data.numpy()

# Flatten each image to a vector
flat_train_data = train_data.reshape(train_data.shape[0], -1)

# Perform SVD for low-rank factorization
U, S, Vt = np.linalg.svd(flat_train_data, full_matrices=False)

# rank ( for example : 16)
rank = 16

# Reconstruct the data using the selected rank
reconstructed_data = U[:, :rank] @ np.diag(S[:rank]) @ Vt[:rank, :]

In [82]:
reconstructed_tensor = torch.Tensor(reconstructed_data)
# Convert the original labels to PyTorch tensors
labels = torch.Tensor(train_dataset.targets.numpy()).long()

# Create a DataLoader with the reconstructed data and labels
train_loader = DataLoader(TensorDataset(reconstructed_tensor, labels), batch_size=64, shuffle=True)

# Train NN on the low-rank factorized data

In [83]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [84]:
class My_NN(nn.Module):

    def __init__(self, ni, nh, no): #ni --> # of input nodes, nh --> # of hidden nodes, no --> # of output nodes
        super().__init__()
        self.layer1 = nn.Linear(ni, nh)
        self.layer2 = nn.Linear(nh, no)

    def forward(self, x):
        x = self.layer1(x)
        x = torch.sigmoid(x)
        x = self.layer2(x)
        return x

In [85]:
net = My_NN(28*28, 500, 10)

In [86]:
lr = 1e-2
num_epochs = 2
opt = optim.SGD(net.parameters(), lr=lr)

for _ in range(num_epochs):
    total_loss = 0
    for batch in train_loader:
        x, y = batch[0], batch[1]

        unique_labels = set(y.numpy())


        logits = net(x)
        loss = F.cross_entropy(logits, y)
        total_loss += loss.item()

        opt.zero_grad()
        loss.backward()
        opt.step()

    print("Epoch {}, Loss: {}".format(_, total_loss))


Epoch 0, Loss: 497.6570819616318
Epoch 1, Loss: 302.81817691773176
